# Principal Component Analysis on MNIST

In this notebook, we get an understanding of **principal component analysis (PCA)** using the familiar MNIST data set of handwritten digits.

### MNIST data

The next few routines check if the MNIST data is already in the current directory; if not, it is downloaded directly from Yann Le Cun's web site. It is then loaded into memory.

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import multivariate_normal
import gzip, sys, os

if sys.version_info[0] == 2:
    from urllib import urlretrieve
else:
    from urllib.request import urlretrieve

In [ ]:
def download(filename, source='http://yann.lecun.com/exdb/mnist/'):
    print("Downloading %s" % filename)
    urlretrieve(source + filename, filename)

def load_mnist_images(filename):
    if not os.path.exists(filename):
        download(filename)
    # Read the inputs in Yann LeCun's binary format.
    with gzip.open(filename, 'rb') as f:
        data = np.frombuffer(f.read(), np.uint8, offset=16)
    data = data.reshape(-1,784)
    return data / np.float32(256)

In [ ]:
train_data = load_mnist_images('../../_data/train-images-idx3-ubyte.gz')

In [ ]:
train_data.shape
n_vector = train_data.shape[1]
np.min(train_data), np.max(train_data)

### Statistics of the data

Principal component analysis chooses projection directions based on the **covariance matrix** of the data.   
This matrix allows us to contrast the effect of picking coordinate directions (i.e. pixels) versus eigenvector directions. In particular:
* *The ith **diagonal entry** of the covariance is the variance in the ith coordinate (the ith pixel).*
* *The ith **eigenvalue** of the covariance matrix is the variance in the direction of the ith eigenvector.*

In [ ]:
# Compute covariance matrix
sigma = np.cov(train_data, rowvar=0, bias=1)

# Compute coordinate-wise variances, in increasing order
coordinate_variances = np.sort(sigma.diagonal())

# Compute variances in eigenvector directions, in increasing order
eigenvector_variances = np.sort(np.linalg.eigvalsh(sigma))

To show the (substantial) benefit of eigenvector projections over coordinate projections, we create a plot that shows the variance lost due to each of these.

For each `k` (projection dimension), we compute:
* How much of the overall variance is lost when we project to the best `k` coordinate directions?
* How much of the overall variance is lost when we project to the top `k` eigenvectors (as in PCA)?

In [ ]:
# Compute fraction of overall variance lost when projecting to k coordinate directions
# cumsum() returns array of cumulative sums, last entry [783] is total sum
total_coordinate_variance = np.cumsum(coordinate_variances)
total_coordinate_variance = total_coordinate_variance/total_coordinate_variance[n_vector-1]

# Compute fraction of overall variance lost when projecting to k eigenvector directions
total_eigenvector_variance = np.cumsum(eigenvector_variances)
total_eigenvector_variance = total_eigenvector_variance/total_eigenvector_variance[n_vector-1]

# Plot these results
plt.plot(np.arange(1, n_vector), total_coordinate_variance[784:0:-1], 'b-', lw=2)
plt.plot(np.arange(1, n_vector), total_eigenvector_variance[784:0:-1], 'r-', lw=2)

plt.xlabel('projection dimension', fontsize=14)
plt.ylabel('fraction of residual variance', fontsize=14)
plt.xlim(0, n_vector)
plt.ylim(0.0, 1.0)
plt.legend(['coordinate directions', 'PCA directions'], fontsize=14)
plt.show();

### Projection and reconstruction

We now get a more *visual* feel for what information is lost during dimensionality reduction.

Suppose we find the PCA projection to `k` dimensions. What is the result of:
* Starting with a handwritten digit in the original (784-dimensional) space
* *Projecting* it down to `k` dimensions
* *Reconstructing* an image in 784-dimensional space from this `k`-dimensional projection?

### Eigenvalues and Eigenvectors of the covariance matrix

 - `numpy.linalg.eigh` returns Eigenvalues and Eigenvectors in order of increasing Eigenvalue.  
 - The Eigenvectors are normalized to unit length and returned as columns of a matrix.

In [ ]:
eigenvalues, eigenvectors = np.linalg.eigh(sigma)

### Projection followed by reconstruction

**`U`** = (784 x `k`) matrix, whose columns are the top `k` eigenvectors;
 - **`transpose(U)`** performs the PCA projection onto the top `k` directions
 - **`U`** reconstructs a point in the original space from its `k` dimensional projection

The dotproduct `U @ transpose(U)` is a (784 x 784) matrix that does a **projection-followed-by-reconstruction**.

In [ ]:
# Function that returns the project-and-reconstruct operations as a single matrix
def projection_and_reconstruction(k):
    U = eigenvectors[:, (n_vector-k):n_vector]
    return U @ U.T

This next routine displays an handwritten digit image given as a 784-dimensional vector. It begins by clipping each entry to lie in the range [0,255]; the images returned after PCA reconstruction might not satisfy this property.

In [ ]:
def show_digit(x, ax, k):
    
    # Clip all entries of x to range [0, 255]
    for i in range(n_vector):
        x[i] = max(0.0, x[i])
        x[i] = min(255.0, x[i])
        
    # Now display
    ax.axis('off')
    ax.imshow(x.reshape((28,28)), cmap=plt.cm.gray)
    ax.set_title('Projection dim.: {}'.format(k))
    return

In [ ]:
def show_effect_of_PCA(x, k_list):
    fig, axes = plt.subplots(1, len(k_list), figsize=(18, 6))
    
    for k, ax in zip(k_list, axes):
        P = projection_and_reconstruction(k)
        show_digit(P @ x, ax, k)

In [ ]:
sample_indices = np.random.permutation(range(n_vector))[:5]

for i in sample_indices:
    show_effect_of_PCA(train_data[i,], [n_vector, 64, 32, 16, 8, 4, 2])
    plt.subplots_adjust(wspace=0.01)

In [ ]:
from sklearn.datasets import fetch_olivetti_faces, fetch_lfw_people
from ipywidgets import interact

### Olivetti Faces

In [ ]:
dataset = fetch_olivetti_faces()
faces = dataset.data
faces.shape

In [ ]:
image_shape = (64, 64)

### Visualise PCA compressed faces

In [ ]:
def projection_and_reconstruction(data, image_shape, k):
    """Return the project-and-reconstruct operations as a single matrix"""
    n_vector = image_shape[0] * image_shape[1]
    sigma = np.cov(data, rowvar=0, bias=1)
    eigenvalues, eigenvectors = np.linalg.eigh(sigma)
    U = eigenvectors[:, (n_vector-k):n_vector]
    return U @ U.T

In [ ]:
def show_face(x, image_shape, ax, k):
    """"""
    # Now display
    ax.axis('off')
    ax.imshow(x.reshape(image_shape), cmap=plt.cm.gray)
    ax.set_title('Projection dim.: {}'.format(k))
    return

In [ ]:
def show_effect_of_PCA(data, x, image_shape, k_list):
    """"""
    fig, axes = plt.subplots(1, len(k_list), figsize=(18, 6))
    
    for k, ax in zip(k_list, axes):
        P = projection_and_reconstruction(data, image_shape, k)
        show_face(P @ x, image_shape, ax, k)

In [ ]:
sample_indices = np.random.permutation(range(faces.shape[0]))[:10]

for i in sample_indices:
    show_effect_of_PCA(faces, faces[i], image_shape, [512, 256, 128, 64, 32, 16, 8, 4])
    plt.subplots_adjust(wspace=0.01)

In [ ]:
faces[0].shape
np.eye(64).shape

In [ ]:
#     Compute the qr factorization of a matrix.
#     Factor the matrix a as qr, where q is orthonormal and r is upper-triangular.
E, _ = np.linalg.qr(B)
E = E.reshape(-1,2)

In [ ]:
def transformation_matrix(B) : 
    
    # The parameter B is a basis is a 2×2 matrix that is passed to the function.
    
    # Use the gsBasis function on bearBasis to get the mirror's orthonormal basis.

#     E = gsBasis(bearBasis)
    
    # Transformation matrix that perform's the mirror's reflection in the mirror's basis.
    TE = np.array([[1, 0],
                   [0, -1]])
    # Combine the matrices E and TE to produce your transformation matrix.
    T = E @ TE @ E.T
    return T


In [ ]:
T = transformation_matrix(np.eye(10))
T.shape

In [ ]:
T @ faces.reshape(-1,64)[0]

In [ ]:
E, _ = np.linalg.qr(faces[0].reshape(-1,64))
E = E.reshape(-1, 64)
E.shape
(E @ np.eye(64) @ E.T)

In [ ]:
dim = 64
face = faces[0].reshape(-1, dim)
t_face = face @ np.eye(dim) @ face.T
plt.imshow(t_face.reshape(-1, 64), cmap=plt.cm.gray);

In [ ]:
show_face_((E @ np.eye(64) @ E.T), (-1,64))